In [1]:
__author__ = 'Pascal Louis <plouis35@gmail.com>'
__version__ = '0.1 alpha'
__keywords__ = ['astronomy', 'astropy', 'ccdproc', 'photutils', 'matplotlib']


# tools for exoplanet / binary stars reduction and analysis
## instruments tested : 
- 
- 